In [57]:
##########################################################################
##### Template code and instructions to scrape Rate My Professor Comments#
##########################################################################
### Step 1: install packages - 
##### packages are necessary to install and load, given that they have the built in functions necessary to run complex tasks. 
## They effectively act as one of the most crucial time saving activities that would otherwise lead to overly long and 
## duplicative scripts. 

## install pkgs 
import sys
!{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install requests #; this code here can be used to install packages on anaconda/jupyter notebook 
### I believe the below should be installed by default 
import requests # web scraping 
from bs4 import BeautifulSoup # for web scraping 
import itertools # for efficient operation of loops 
import pandas as pd # necessary for reading in, creating, and manipulating data frames 
import csv ## for importing/exporting csvs 
import glob ## for finding files in path
import re


In [17]:
### Step 2: Read in the csv with the data of interest 
## note: This will be the section that you update manually the most. The rest should be automated 

faculty_csv = pd.read_csv('faculty characteristics web scraping_UNC.csv')
urls = faculty_csv['link_rmp']


## url to use ; will be the profs page, with the numeric code being what changes; should be changed manually 

#url = ['https://www.ratemyprofessors.com/professor/1190096','https://www.ratemyprofessors.com/professor/2070585']

## college 
college = faculty_csv['college']

##prof last name 
prof_lastname = faculty_csv['prof_lastname']

# prof first name 
prof_firstname = faculty_csv['prof_firstname']

## note: you will want to manually seach the rate my professor website for the professor of interest. From there, you will 
# be able to grab the url for the professor of interest, which will be the last numeric digits of the url that will vary. 
#url
faculty_csv

,college,prof_firstname,prof_lastname,program,male,non_white,link_rmp,notes
0,UNIVERSITY OF NORTH CAROLINA,DALE,WHITTINGTON,ENVIRONMENTAL SCIENCES & ENGINEERING,1,0,https://www.ratemyprofessors.com/professor/100...,NaN
1,UNIVERSITY OF NORTH CAROLINA,COURTNEY,WOODS,ENVIRONMENTAL SCIENCES & ENGINEERING,0,1,https://www.ratemyprofessors.com/professor/225...,NaN
2,UNIVERSITY OF NORTH CAROLINA,KARI,NORTH,EPIDEMIOLOGY,0,0,https://www.ratemyprofessors.com/professor/212...,NaN
3,UNIVERSITY OF NORTH CAROLINA,SUZANNE,MAMAN,HEALTH BEHAVIOR,0,0,https://www.ratemyprofessors.com/professor/286...,Ratings are for Health Science
4,UNIVERSITY OF NORTH CAROLINA,BENJAMIN,MEIER,HEALTH POLICY & MANAGEMENT,1,0,https://www.ratemyprofessors.com/professor/143...,Ratings are for Public Policy
...,...,...,...,...,...,...,...,...
97,UNIVERSITY OF NORTH CAROLINA,TED,MOUW,SOCIOLOGY,1,0,https://www.ratemyprofessors.com/professor/379594,NaN
98,UNIVERSITY OF NORTH CAROLINA,LISA,PEARCE,SOCIOLOGY,0,0,https://www.ratemyprofessors.com/professor/221...,NaN
99,UNIVERSITY OF NORTH CAROLINA,ALEXANDREA,RAVENELLE,SOCIOLOGY,0,0,https://www.ratemyprofessors.com/professor/259...,NaN
100,UNIVERSITY OF NORTH CAROLINA,JESSICA,SU,SOCIOLOGY,0,0,https://www.ratemyprofessors.com/professor/252...,NaN


In [18]:
### Step 3: run nested loop with these data

for j, element in enumerate(urls):
    temp_page = requests.get(element) ## grabbing the page from the url, which is stored in the jth row of the url list
    soup = BeautifulSoup(temp_page.text, "html.parser") # parse the html for the jth pulled page 
    prof_comments = soup.findAll("div", {"class": "Comments__StyledComments-dzzyvm-0 gRjWel" }) # grab the comments 
    prof_score = soup.findAll("div", {"class": "CardNumRating__CardNumRatingNumber-sc-17t4b9u-2" }) # grab scores 
    prof_class = soup.findAll("div", {"class": "RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt" }) # just take the odds
    ## split up into difficulty, quality and class num 
    prof_dif = prof_score[1::2]
    prof_qual = prof_score[0::2]
    prof_class = prof_class[1::2]
    ### now a new loop to store the scraped data from the jth row into the new temp df 
    d1 = []
    for i, element in enumerate(prof_qual):
        d1.append({
            'row': i+1,
            'quality_of_class': prof_qual[i].get_text(),
            "difficulty_of_class": prof_dif[i].get_text(),
            "class_code": prof_class[i].get_text(),
            "college": college[j], #jth lvl, not ith level, variable 
            "prof_firstname": prof_firstname[j],            
            "prof_lastname": prof_lastname[j], # same logic as above, now for the prof's last name         
            'comment': prof_comments[i].get_text() #ith level again 
        }
        )
    df = pd.DataFrame(d1)
    ### now save the file 
    save_name = "scraped_data/rmp_data" +"_"+college[j] +"_"+prof_lastname[j]+"_"+prof_firstname[j]+".csv"
    df.to_csv(save_name, index=False)







In [24]:
#### compile all of the files in the directory ## 
scraped_data_files =  glob.glob("scraped_data/*")
scraped_data_files

['scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BOWEN_RACHEL.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_ABRAMOWITZ_JONATHAN.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_ALDRICH_HOWARD.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_ALGOE_SARA.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_ANDERSON_ASHLEY.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_ANDREWS_KENNETH.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_ARNOLD_JENNIFER.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BALLARD-ROSA_CAMERON.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BAPAT_NAVIN.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BARDONE-CONE_ANNA.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BASSI_ANNA.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BAUCOM_DONALD.csv',
 'scraped_data\\rmp_data_UNIVERSITY OF NORTH CAROLINA_BAUER_DANIEL.csv',
 'scraped_data\\rmp_data_UNIVER

In [42]:
master_unc = pd.DataFrame()
  
for element in scraped_data_files:
    temp_df = pd.read_csv(element)
    master_unc = master_unc.append(temp_df)

    
           
        
    
    
    
    
    

C:\Users\j-curiel\AppData\Local\Temp\ipykernel_13212\3721422271.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_unc = master_unc.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_13212\3721422271.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_unc = master_unc.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_13212\3721422271.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_unc = master_unc.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_13212\3721422271.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_unc = master_unc.append(temp_df)
C:\Users\j-curiel\Ap

In [43]:
master_unc

,row,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment
0,1,5.0,1.0,POL1200,OHIO STATE UNIVERSITY,RACHEL,BOWEN,There was a lot of reading but it was not nece...
1,2,5.0,1.0,POLSCI1300,OHIO STATE UNIVERSITY,RACHEL,BOWEN,Dr. Bowen is a really great professor. Her cla...
2,3,3.0,3.0,POLITSC1100,OHIO STATE UNIVERSITY,RACHEL,BOWEN,Professor Bowen is a decent professor at OSU M...
3,4,4.0,2.0,POL1200,OHIO STATE UNIVERSITY,RACHEL,BOWEN,Pretty good professor. She's very liberal but ...
4,5,5.0,2.0,POLITSC1100,OHIO STATE UNIVERSITY,RACHEL,BOWEN,Gives great feedback and clearly outlines what...
...,...,...,...,...,...,...,...,...
9,10,4.5,2.0,PSYC210,UNIVERSITY OF NORTH CAROLINA,ERIC,YOUNGSTROM,This dude is goofy as hell. He does his best ...
10,11,4.5,1.0,PSYC210,UNIVERSITY OF NORTH CAROLINA,ERIC,YOUNGSTROM,He did a great job of making this a good class...
11,12,4.5,4.0,PSYC210,UNIVERSITY OF NORTH CAROLINA,ERIC,YOUNGSTROM,Best Prof Ever! Makes Stats fun and very good ...
12,13,4.0,2.0,PSYC210,UNIVERSITY OF NORTH CAROLINA,ERIC,YOUNGSTROM,"Wonderful guy!!! He made stats painless, which..."


In [48]:
#### now create the empty columns
### outrage vars 
master_unc["out_misrep"] = 0
master_unc["out_emo_lang"] = 0
master_unc["out_ideo_extrem"] = 0
master_unc["out_slip_slope"] = 0
master_unc["out_insult"] = 0
master_unc["out_mock_sarc"] = 0
master_unc["out_threat"] = 0
## personal attacks 
master_unc["pa_dress"] = 0
master_unc["pa_appear"] = 0
master_unc["pa_attit"] = 0
master_unc["pa_interact"] = 0
master_unc["pa_humor"] = 0
master_unc["pa_other"] = 0
### predjudice and bigotry 
master_unc["pb_sex"] = 0
master_unc["pb_lgbtq"] = 0
master_unc["pb_race"] = 0
master_unc["pb_origin"] = 0
master_unc["pb_nuero_div"] = 0
master_unc["pb_phys_able"] = 0
master_unc["pb_pol_affil"] = 0
### the comments that might be helpful 
master_unc["complex"] = 0
master_unc["constructive"] = 0
master_unc["reflective"] = 0









In [49]:
### now export 
master_unc.to_csv("master_unc_data_comments.csv", index=False)


In [55]:
#### Import the OSU data 
faculty_csv = pd.read_csv('faculty characteristics web scraping_OSU2.csv')

faculty_csv=faculty_csv.loc[faculty_csv['link_rmp'] != "DNE"]

urls = faculty_csv['link_rmp']


## url to use ; will be the profs page, with the numeric code being what changes; should be changed manually 

#url = ['https://www.ratemyprofessors.com/professor/1190096','https://www.ratemyprofessors.com/professor/2070585']

## college 
college = faculty_csv['college']

##prof last name 
prof_lastname = faculty_csv['prof_lastname']

# prof first name 
prof_firstname = faculty_csv['prof_firstname']

## note: you will want to manually seach the rate my professor website for the professor of interest. From there, you will 
# be able to grab the url for the professor of interest, which will be the last numeric digits of the url that will vary. 
#url
faculty_csv




,college,prof_firstname,prof_lastname,program,male,non_white,link_rmp,NOTES
0,OHIO STATE UNIVERSITY,NAOMI,ADANIYA,PUBLIC HEALTH,0,1,https://www.ratemyprofessors.com/professor/207...,NaN
1,OHIO STATE UNIVERSITY,SARA,ANDERSON,PUBLIC HEALTH,0,0,https://www.ratemyprofessors.com/professor/272...,NaN
2,OHIO STATE UNIVERSITY,REBECCA,ANDRIDGE,PUBLIC HEALTH,0,1,https://www.ratemyprofessors.com/professor/240...,NaN
3,OHIO STATE UNIVERSITY,KELLIE,ARCHER,PUBLIC HEALTH,0,0,https://www.ratemyprofessors.com/professor/245...,NaN
4,OHIO STATE UNIVERSITY,GIA,BARBOZA-SALERNO,PUBLIC HEALTH,0,0,https://www.ratemyprofessors.com/professor/287...,NaN
...,...,...,...,...,...,...,...,...
225,OHIO STATE UNIVERSITY,JOHN,MUELLER,POLITICAL SCIENCE,1,0,https://www.ratemyprofessors.com/professor/175682,NaN
226,OHIO STATE UNIVERSITY,RANDALL,RIPLEY,POLITICAL SCIENCE,1,0,https://www.ratemyprofessors.com/professor/811378,NaN
227,OHIO STATE UNIVERSITY,GOLDIE,SHABAD,POLITICAL SCIENCE,0,0,https://www.ratemyprofessors.com/professor/93027,NaN
228,OHIO STATE UNIVERSITY,ELLIOT,SLOTNICK,POLITICAL SCIENCE,1,0,https://www.ratemyprofessors.com/professor/51211,NaN


In [54]:
len(urls)

230

In [56]:
### Step 3: run nested loop with these data

for j, element in enumerate(urls):
    temp_page = requests.get(element) ## grabbing the page from the url, which is stored in the jth row of the url list
    soup = BeautifulSoup(temp_page.text, "html.parser") # parse the html for the jth pulled page 
    prof_comments = soup.findAll("div", {"class": "Comments__StyledComments-dzzyvm-0 gRjWel" }) # grab the comments 
    prof_score = soup.findAll("div", {"class": "CardNumRating__CardNumRatingNumber-sc-17t4b9u-2" }) # grab scores 
    prof_class = soup.findAll("div", {"class": "RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt" }) # just take the odds
    ## split up into difficulty, quality and class num 
    prof_dif = prof_score[1::2]
    prof_qual = prof_score[0::2]
    prof_class = prof_class[1::2]
    ### now a new loop to store the scraped data from the jth row into the new temp df 
    d1 = []
    for i, element in enumerate(prof_qual):
        d1.append({
            'row': i+1,
            'quality_of_class': prof_qual[i].get_text(),
            "difficulty_of_class": prof_dif[i].get_text(),
            "class_code": prof_class[i].get_text(),
            "college": college[j], #jth lvl, not ith level, variable 
            "prof_firstname": prof_firstname[j],            
            "prof_lastname": prof_lastname[j], # same logic as above, now for the prof's last name         
            'comment': prof_comments[i].get_text() #ith level again 
        }
        )
    df = pd.DataFrame(d1)
    ### now save the file 
    save_name = "scraped_data/rmp_data" +"_"+college[j] +"_"+prof_lastname[j]+"_"+prof_firstname[j]+".csv"
    df.to_csv(save_name, index=False)







In [59]:
### Now let's grab only the OSU files 

#### compile all of the files in the directory ## 
scraped_data_files =  glob.glob("scraped_data/*")
scraped_data_files_osu = [x for x in scraped_data_files if re.findall("OHIO STATE",x)] # grab files that have "zest" in title
scraped_data_files_osu

['scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ACS_ALEX.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ADANIYA_NAOMI.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ANDERSON_SARA.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ANDRIDGE_REBECCA.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ANGEL_WILLIAM.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ARCHER_KELLIE.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ARKES_HAL.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_ASHER_HERB.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BAKER_JEREMY.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BARBOZA-SALERNO_GIA.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BASTA_NICHOLAS.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BAUM_LAWRENCE.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BECK_PAUL.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BECK_STEVEN.csv',
 'scraped_data\\rmp_data_OHIO STATE UNIVERSITY_BELLAIR_PAUL.csv',
 'scr

In [60]:
### now loop in and read the data 
master_osu = pd.DataFrame()
  
for element in scraped_data_files_osu:
    temp_df = pd.read_csv(element)
    master_osu = master_osu.append(temp_df)

    
           
        
    
    
    
    
    

C:\Users\j-curiel\AppData\Local\Temp\ipykernel_13212\1339760224.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_13212\1339760224.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_13212\1339760224.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\AppData\Local\Temp\ipykernel_13212\1339760224.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  master_osu = master_osu.append(temp_df)
C:\Users\j-curiel\Ap

EmptyDataError: No columns to parse from file

In [61]:
master_osu

,row,quality_of_class,difficulty_of_class,class_code,college,prof_firstname,prof_lastname,comment
0,1,2.0,4.0,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,"Only graded on 4 assignments (30% Midterm, 30%..."
1,2,3.0,4.0,POLITSC3500,OHIO STATE UNIVERSITY,ALEX,ACS,"Final grade is only based on two exams, readin..."
2,3,4.0,1.0,POLITSC1100,OHIO STATE UNIVERSITY,ALEX,ACS,Class was super easy. One reading quiz a week ...
3,4,4.0,2.0,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,"Lecture could be dry at times, but I still lik..."
4,5,5.0,3.0,POLITSC3115,OHIO STATE UNIVERSITY,ALEX,ACS,"Alex was an excellent lecturer. Insightful, ev..."
...,...,...,...,...,...,...,...,...
6,7,5.0,1.0,SOCIOL2309,OHIO STATE UNIVERSITY,JACI,BOWE,There was one required final paper and 3 exams...
7,8,5.0,1.0,SOCIOL2309,OHIO STATE UNIVERSITY,JACI,BOWE,"Professor Bowe was amazing! She's an attorney,..."
8,9,4.0,1.0,SOCIOL2309,OHIO STATE UNIVERSITY,JACI,BOWE,Online class so I didn't interact with her muc...
9,10,5.0,1.0,SOCIOL4511,OHIO STATE UNIVERSITY,JACI,BOWE,Such an easy A and she is honestly the best! H...


In [ ]:
#### now create the empty columns
### outrage vars 
master_osu["out_misrep"] = 0
master_osu["out_emo_lang"] = 0
master_osu["out_ideo_extrem"] = 0
master_osu["out_slip_slope"] = 0
master_osu["out_insult"] = 0
master_osu["out_mock_sarc"] = 0
master_osu["out_threat"] = 0
## personal attacks 
master_osu["pa_dress"] = 0
master_osu["pa_appear"] = 0
master_osu["pa_attit"] = 0
master_osu["pa_interact"] = 0
master_osu["pa_humor"] = 0
master_osu["pa_other"] = 0
### predjudice and bigotry 
master_osu["pb_sex"] = 0
master_osu["pb_lgbtq"] = 0
master_osu["pb_race"] = 0
master_osu["pb_origin"] = 0
master_osu["pb_nuero_div"] = 0
master_osu["pb_phys_able"] = 0
master_osu["pb_pol_affil"] = 0
### the comments that might be helpful 
master_osu["complex"] = 0
master_osu["constructive"] = 0
master_osu["reflective"] = 0







